# 🛒 E-commerce Funnel Analysis with SQL

This notebook demonstrates how to analyze the user journey funnel (view → cart → purchase) using SQL, based on the Kaggle e-commerce dataset.

In [3]:
# Load data using SQLite
import pandas as pd
import sqlite3

# Load the dataset into a DataFrame
df = pd.read_csv('2019-Nov.csv')

# Create SQLite in-memory DB
conn = sqlite3.connect(':memory:')
df.to_sql('user_events', conn, index=False, if_exists='replace')

# Check schema
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904.0,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790.0,8e5f4f83-366c-4f70-860e-ca7417414283
2,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,NaN,creed,28.31,561587266.0,755422e7-9040-477b-9bd2-6a6e8fd97387
3,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591.0,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
4,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683.0,313628f1-68b8-460d-84f6-cec7a8796ef2


## 🔍 SQL Query: Session-level Funnel Construction

In [5]:
query = '''
WITH session_funnel AS (
  SELECT 
    user_id,
    user_session,
    MAX(CASE WHEN event_type = 'view' THEN 1 ELSE 0 END) AS viewed,
    MAX(CASE WHEN event_type = 'cart' THEN 1 ELSE 0 END) AS carted,
    MAX(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) AS purchased
  FROM user_events
  GROUP BY user_id, user_session
)
SELECT 
  COUNT(*) AS total_sessions,
  SUM(viewed) AS view_sessions,
  SUM(carted) AS cart_sessions,
  SUM(purchased) AS purchase_sessions,
  ROUND(1.0 * SUM(carted) / SUM(viewed), 2) AS view_to_cart_rate,
  ROUND(1.0 * SUM(purchased) / SUM(carted), 2) AS cart_to_purchase_rate
FROM session_funnel;
'''

pd.read_sql_query(query, conn)

,total_sessions,view_sessions,cart_sessions,purchase_sessions,view_to_cart_rate,cart_to_purchase_rate
0,2240882,2240471,80728,130307,0.04,1.61
